In [ ]:
!pip -qqq install bitsandbytes accelerate
!pip install -q -U transformers==4.30
!pip install -q -U git+https://github.com/huggingface/peft.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import os
import torch
import pandas as pd
from transformers import LlamaTokenizer, LlamaForCausalLM
import accelerate
import bitsandbytes
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -c "from accelerate.utils import write_basic_config; write_basic_config(mixed_precision='fp16')"

In [ ]:
model = LlamaForCausalLM.from_pretrained(
    "maritaca-ai/sabia-7b",
    device_map="auto",
    low_cpu_mem_usage=True,
    load_in_8bit=True,  # Requires pip install bitsandbytes
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at maritaca-ai/sabia-7b and are newly initialized: ['model.layers.9.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.30.self_attn.rotary_emb.inv_freq', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.23.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
!gdown 1lKMjhZFDnuKvvmN7zxDe9i1JTWwNVVrl
!gdown 1lQlz4S5KSj3zcMVEp6RYXOJJ4l2fcBst
!gdown 1lwqkm7wYsGhsBE_N5iB3FEbmexZQjCEz
!gdown 1mAFqzxyYdAx0twEW17mekM0NXvqg8FDg
!gdown 1mCs8NpUTDIYSTdlWmhLIT8x81DHk5yIV
!gdown 1mFLhjzsBc6VkRK4aInmYzpJ0ml-Fn3Li
!gdown 1mFaaMK2EefQ8MCNJJAKczfbLmGWpOfe8

Downloading...
From: https://drive.google.com/uc?id=1lKMjhZFDnuKvvmN7zxDe9i1JTWwNVVrl
To: /content/caras.brwac.csv
100% 28.4M/28.4M [00:00<00:00, 32.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1lQlz4S5KSj3zcMVEp6RYXOJJ4l2fcBst
To: /content/educacao.brwac.csv
100% 15.2M/15.2M [00:00<00:00, 36.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1lwqkm7wYsGhsBE_N5iB3FEbmexZQjCEz
To: /content/esporte.brwac.csv
100% 30.6M/30.6M [00:00<00:00, 35.4MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1mAFqzxyYdAx0twEW17mekM0NXvqg8FDg
From (redirected): https://drive.google.com/uc?id=1mAFqzxyYdAx0twEW17mekM0NXvqg8FDg&confirm=t&uuid=1d48c5d8-49cc-486e-bd7b-c01d0b78c37c
To: /content/folha.brwac.csv
100% 137M/137M [00:04<00:00, 30.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mCs8NpUTDIYSTdlWmhLIT8x81DHk5yIV
To: /content/jogos.brwac.csv
100% 7.65M/7.65M [00:00<00:00, 20.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mFLhjzsBc6VkRK4aI

In [ ]:
caras_df = pd.read_csv("caras.brwac.csv", nrows=1000)
esporte_df = pd.read_csv("esporte.brwac.csv", nrows=1000)
educacao_df = pd.read_csv("educacao.brwac.csv", nrows=1000)
folha_df = pd.read_csv("folha.brwac.csv", nrows=1000)
jogos_df = pd.read_csv("jogos.brwac.csv", nrows=1000)
noticias_df = pd.read_csv("noticias.brwac.csv", nrows=1000)
tecnologia_df = pd.read_csv("tecnologia.brwac.csv")

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("maritaca-ai/sabia-7b")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    outputs = tokenizer(examples, padding='max_length',
                        truncation=True, max_length=500)

    # Usar input_ids como labels
    outputs["labels"] = outputs["input_ids"].copy()
    return outputs

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
caras_df = caras_df['text'].map(preprocess_function)
esporte_df = esporte_df['text'].map(preprocess_function)
educacao_df = educacao_df['text'].map(preprocess_function)
folha_df = folha_df['text'].map(preprocess_function)
jogos_df = jogos_df['text'].map(preprocess_function)
noticias_df = noticias_df['text'].map(preprocess_function)

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=True,
    r=8,
    lora_alpha=32,
    bias="none",
    lora_dropout=0.1
)

In [ ]:
# Aplicar Adapters ao modelo
lora_model = get_peft_model(model, peft_config)
lora_model = prepare_model_for_kbit_training(lora_model)

In [ ]:
# Configurar os parâmetros de treinamento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
def treinar(dataset, name: str):
    train_dataset, test_dataset = train_test_split(
        dataset.reset_index(drop=True), test_size=0.2, random_state=42)
    train_dataset = train_dataset.tolist()
    test_dataset = test_dataset.tolist()

    trainer = Trainer(
        model=lora_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
    )

    trainer.train()

    save_path = f"/content/drive/MyDrive/krules-results/sabia/results/{name}"
    os.makedirs(save_path, exist_ok=True)
    model.save_pretrained(save_path)
    return trainer

In [ ]:
caras = treinar(caras_df, "caras")
esporte = treinar(esporte_df, "esporte")
educacao = treinar(educacao_df, "educacao")
folha = treinar(folha_df, "folha")
jogos = treinar(jogos_df, "jogos")
noticias = treinar(noticias_df, "noticias")
tecnologia = treinar(tecnologia_df, "tecnologia")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.fl

Epoch,Training Loss,Validation Loss
1,7.581200,8.059678
2,7.549000,8.059678
3,7.437500,8.059678


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 d

Epoch,Training Loss,Validation Loss
1,2.902300,2.477062
2,2.766400,2.477062
3,2.793500,2.477062


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 d

...

Epoch,Training Loss,Validation Loss
1,2.272800,2.391978
2,2.399700,2.391978
3,2.377400,2.391978


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 d

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,14.798300,14.824635
2,14.630700,14.824635
3,14.877600,14.824635


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 d